In [1]:
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [2]:
num_classes = 101

In [3]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [4]:
val_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [5]:
train_dataset = datasets.Food101(root="./data", split="train", transform=train_transforms, download=True)
val_dataset = datasets.Food101(root="./data", split="test", transform=val_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

100%|██████████| 5.00G/5.00G [02:44<00:00, 30.4MB/s]


In [6]:
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 184MB/s]


In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [9]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss, correct = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
    
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [10]:
def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * images.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
    
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [11]:
for epoch in range(10):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    
    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc*100:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.2f}%")

Epoch 1:
  Train Loss: 1.7042, Accuracy: 57.89%
  Val   Loss: 0.9457, Accuracy: 73.89%
Epoch 2:
  Train Loss: 1.0740, Accuracy: 71.54%
  Val   Loss: 0.8388, Accuracy: 76.75%
Epoch 3:
  Train Loss: 0.8913, Accuracy: 76.02%
  Val   Loss: 0.7437, Accuracy: 79.56%
Epoch 4:
  Train Loss: 0.7785, Accuracy: 78.76%
  Val   Loss: 0.7663, Accuracy: 78.95%
Epoch 5:
  Train Loss: 0.6859, Accuracy: 80.87%
  Val   Loss: 0.7590, Accuracy: 79.10%
Epoch 6:
  Train Loss: 0.6137, Accuracy: 82.64%
  Val   Loss: 0.7046, Accuracy: 80.95%
Epoch 7:
  Train Loss: 0.5541, Accuracy: 84.26%
  Val   Loss: 0.7171, Accuracy: 80.72%
Epoch 8:
  Train Loss: 0.4944, Accuracy: 85.80%
  Val   Loss: 0.7090, Accuracy: 81.13%
Epoch 9:
  Train Loss: 0.4570, Accuracy: 86.75%
  Val   Loss: 0.7412, Accuracy: 80.52%
Epoch 10:
  Train Loss: 0.4110, Accuracy: 87.84%
  Val   Loss: 0.7137, Accuracy: 81.74%


In [12]:
torch.save(model.state_dict(), '/kaggle/working/Food101pred.pth')

In [13]:
import joblib
file_name = 'Food101pred.pth' 
joblib.dump(model, file_name)

['Food101pred.pth']